<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dados" data-toc-modified-id="Dados-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dados</a></span></li><li><span><a href="#Gráficos" data-toc-modified-id="Gráficos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gráficos</a></span></li><li><span><a href="#HTML-Figure" data-toc-modified-id="HTML-Figure-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>HTML Figure</a></span></li></ul></div>

In [23]:
from pyhvr import *
import datetime
import matplotlib.font_manager
%config Completer.use_jedi = False

In [24]:
loadstd()

In [ ]:
# DT_NOTIFIC: Data de preenchimento da ficha de notificação.
# DT_SIN_PRI: Data de 1ºs sintomas da SRAG
# CO_MUN_NOT: Município onde está localizada a Unidade Sentinela que realizou anotificação.
# CO_RG_RESI: Regional de Saúde onde está localizado o Município de residência do paciente
# CO_MUN_RES: Município de residência do paciente
# FEBRE: Sinais e Sintomas/Febre
# DT_DIGITA: Data de inclusão do registro no sistema

srag_df = pd.read_csv('INFLUD-30-06-2020.csv',sep=';', encoding='cp860')
srag_mga = srag_df[srag_df['CO_RG_RESI']==1369][['DT_NOTIFIC', 'DT_SIN_PRI', 'CO_MUN_NOT', 'CO_RG_RESI', 'CO_MUN_RES', 'FEBRE', 'DT_DIGITA', 'PCR_SARS2']]
srag_mga = srag_mga[srag_mga.FEBRE.isin([1,2])]
srag_mga = srag_mga[srag_mga.PCR_SARS2 == 1]

In [21]:
max_delay = 14
onset = pd.to_datetime(srag_mga['DT_SIN_PRI'], format='%d/%m/%Y')
confirmed = pd.to_datetime(srag_mga['DT_NOTIFIC'], format='%d/%m/%Y')
delays = (confirmed - onset).dt.days
delays = delays[delays<max_delay].reset_index(drop=True)
p_delay = delays.value_counts().sort_index()
new_range = np.arange(0, p_delay.index.max() + 1)
p_delay = p_delay.reindex(new_range, fill_value=0)
p_delay /= p_delay.sum()
INCUBATION_DAYS=5
p_delay = (
    pd.Series(np.zeros(INCUBATION_DAYS))
    .append(p_delay, ignore_index=True)
    .rename("p_delay")
)

In [25]:
p_delay.to_csv('data/p_delay.csv', index=False)